In [2]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [3]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [4]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [5]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [6]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [7]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [8]:
df['loc']

1     (9349 Collins Ave Apt 402, Surfside, FL 33154,...
2     (10201 Collins Ave Unit 2402, Bal Harbour, FL ...
3     (1 Collins Ave Unit 406, Miami Beach, FL 33139...
4     (9705 Collins Ave #403n, Bal Harbour, FL 33154...
5     (3739 Collins Ave, Miami Beach, FL 33140, USA,...
6     (4775 Collins Ave APT 3503, Miami Beach, FL 33...
7     (15701 Collins Ave #3703, North Miami Beach, F...
8     (1331 Brickell Bay Dr APT 3405, Miami, FL 3313...
9     (881 Ocean Dr #10a, Key Biscayne, FL 33149, US...
10    (17121 Collins Ave APT 1608, Sunny Isles Beach...
Name: loc, dtype: object

In [9]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [10]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Listing Broker,Buyer Agent,Buyer Broker,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Fendi Chateau\r9349 Collins Ave 402\rSurfside,3/24/2022,5/25/2022,14,"$9,500,000.00",3453,"$2,751.23",Romina Grinberg,Brown Harris Stevens,Ximena Penuela,Clarte Realty,Fendi Chateau 9349 Collins Ave 402 Surfside,Fendi Chateau,9349 Collins Ave 402 Surfside,"(9349 Collins Ave Apt 402, Surfside, FL 33154,...","(25.8830205, -80.1216893, 0.0)",25.883021,-80.121689,0.0
2,Oceana Bal Harbour Condo\r10201 Collins Ave 24...,11/1/2021,5/24/2022,146,"$8,500,000.00",3450,"$2,463.77",Matias Derito,"HQ Int'l Realty Group, LLC",Bruno Junqueira,Coldwell Banker Realty,Oceana Bal Harbour Condo 10201 Collins Ave 240...,Oceana Bal Harbour Condo,10201 Collins Ave 2402 Bal Harbour,"(10201 Collins Ave Unit 2402, Bal Harbour, FL ...","(25.8951782, -80.1231506, 0.0)",25.895178,-80.123151,0.0
3,One Ocean\r1 Collins Ave 406\rMiami Beach,4/18/2022,5/24/2022,10,"$6,995,000.00",3337,"$2,096.19",Dina Goldentayer,Douglas Elliman,Marco Tine,Coldwell Banker Realty,One Ocean 1 Collins Ave 406 Miami Beach,One Ocean,1 Collins Ave 406 Miami Beach,"(1 Collins Ave Unit 406, Miami Beach, FL 33139...","(25.768079, -80.1339531, 0.0)",25.768079,-80.133953,0.0
4,St Regis Bal Harbour\r9705 Collins Ave 403N\rB...,9/2/2021,5/24/2022,204,"$5,100,000.00",2840,"$1,795.77",Ricardo Montalvan,"Weichert Realtors, NexGen",NON-MLS MEMBER,Non-MLS Member,St Regis Bal Harbour 9705 Collins Ave 403N Bal...,St Regis Bal Harbour,9705 Collins Ave 403N Bal Harbour,"(9705 Collins Ave #403n, Bal Harbour, FL 33154...","(25.8892229, -80.1228812, 0.0)",25.889223,-80.122881,0.0
5,The Caribbean Condo\r3739 Collins Ave N-601/3\...,1/18/2022,5/26/2022,120,"$4,250,000.00",3357,"$1,266.01",Michael Duchon,The Corcoran Group,Eloy Carmenate,The Corcoran Group,The Caribbean Condo 3739 Collins Ave N-601/3 M...,The Caribbean Condo,3739 Collins Ave N-601/3 Miami Beach,"(3739 Collins Ave, Miami Beach, FL 33140, USA,...","(25.8105453, -80.1225323, 0.0)",25.810545,-80.122532,0.0
6,Green Diamond\r4775 Collins Ave 3503\rMiami Beach,4/12/2022,5/23/2022,8,"$3,265,500.00",1980,"$1,649.24",Michele Redlich,Coldwell Banker Realty,Menachem Fellig,"Compass Florida, LLC",Green Diamond 4775 Collins Ave 3503 Miami Beach,Green Diamond,4775 Collins Ave 3503 Miami Beach,"(4775 Collins Ave APT 3503, Miami Beach, FL 33...","(25.8227293, -80.1226745, 0.0)",25.822729,-80.122675,0.0
7,Ritz Carlton\r15701 Collins Ave 3703\rSunny Is...,10/8/2021,5/24/2022,194,"$2,900,000.00",1447,"$2,004.15",Giulietta Ulloa,BHHS EWM Realty,Richard Klassman,"Winston Towers Realty, Inc",Ritz Carlton 15701 Collins Ave 3703 Sunny Isle...,Ritz Carlton,15701 Collins Ave 3703 Sunny Isles Beach,"(15701 Collins Ave #3703, North Miami Beach, F...","(25.9220146, -80.1217034, 0.0)",25.922015,-80.121703,0.0
8,Jade Residences Brickell\r1331 Brickell Bay Dr...,4/1/2022,5/26/2022,39,"$2,575,000.00",2130,"$1,208.92",Silvana Giuffrida,London Real Estate Co,Christopher Richard,Re/Max Advance Realty,Jade Residences Brickell 1331 Brickell Bay Dr ...,Jade Residences Brickell,1331 Brickell Bay Dr 3405 Miami,"(1331 Brickell Bay Dr APT 3405, Miami, FL 3313...","(25.7603363, -80.1893438, 0.0)",25.760336,-80.189344,0.0
9,Casa del Mar\r881 Ocean Dr 10A\rKey Biscayne,3/3/2022,5/23/2022,15,"$2,500,000.00",3120,$801.28,Karina Asturias Reguera,"Compass Florida, LLC",Dolores Urdapilleta,"Urdapilleta Real Estate, LLC",Casa del Mar 881 Ocean Dr 10A Key Biscayne,Casa del Mar,881 Ocean Dr 10A Key Biscayne,"(881 Ocean Dr #10a, Key Biscayne, FL 33149, US...","(25.685686, -80.15729309999999, 0.0)",25.685686,-80.157293,0.0
10,J0ade Ocean Condo\r17121 Collins Ave 1608\rSun...,3/3/2022,5/24/2022,19,"$2,275,000.00",1956,"$1,163.09",Richard Goihman,Douglas Elliman,Vanesa Seijas,Urban Development & Investments LLC,J0ade Ocean Condo 17121 Collins Ave 1608 Sunny...,J,0ade Ocean Condo 17121 Collin

Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [11]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [12]:
df.at[10,'building_name']=('Jade Ocean')
df.at[10,'address_only']=('17121 Collins Ave 1608')

df.at[5,'Agent']=('Mick Duchon')
df.at[6,'Buyer Agent']=('Mendel Fellig')

In [13]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Listing Broker',
       'Buyer Agent', 'Buyer Broker', 'geo_address', 'building_name',
       'address_only', 'loc', 'point', 'lat', 'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('May 22nd - May 28th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['price_per_sqft'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}{info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')